## Biot's equation

This notebooks aims at testing the effect of different boundary conditions for Biot's equation on a simple 1D intervall mesh. the quasi-static equations for Biot's model are:

$$
- \nabla \cdot 2 \mu \epsilon (u) - \nabla \lambda \nabla u I + \alpha \nabla p = f \quad \text{in $\Omega$ (momentum equation)}
$$
$$
c \frac{\partial p}{\partial t} + \alpha \frac{\nabla \cdot u}{\partial t} - \nabla \cdot (K \nabla p) = g \quad \text{ in $\Omega$ (continuity equation)}
$$

Suitable boundary conditions might be either No-flux or Dirichlet BC for the pressure and a total stress BC for the displacement:

$$
(2 \mu \epsilon (u) + \lambda \nabla u I - \alpha p I) \cdot n = - p_{obs} n \quad \text{on $\partial \Omega$}
$$

In [ ]:
%load_ext autoreload
%autoreload 2
from fenics import *
import matplotlib.pyplot as plt
import numpy as np
from fenics_adjoint import *
from braininversion.BiotSolver import solve_biot
from braininversion.PlottingHelper import (plot_pressures_and_forces_timeslice, 
                            plot_pressures_and_forces_cross_section,
                            extract_cross_section, style_dict)


T = 1.0           # final time
num_steps = 20    # number of time steps
nx = 100
dt = T / num_steps
times = np.linspace(dt, T, num_steps)

# material parameter
kappa = 1e-17       # permeability 15*(1e-9)**2
visc = 0.8*1e-3     # viscocity 
K = kappa/visc      # hydraulic conductivity
c = 2*1e-8         # storage coefficent
alpha = 1.0         # Biot-Willis coefficient

# Biot material parameters
E = 1500.0          # Young modulus
nu = 0.479         # Poisson ratio

material_parameter = dict()
material_parameter["c"] = c
material_parameter["K"] = K
material_parameter["lmbda"] = nu*E/((1.0-2.0*nu)*(1.0+nu)) 
material_parameter["mu"] = E/(2.0*(1.0+nu))
material_parameter["alpha"] = alpha

mesh = Mesh(IntervalMesh(nx,0.0,0.1))
bm = MeshFunction("size_t", mesh, 0)
left = CompiledSubDomain("near(x[0], 0.0)")
right = CompiledSubDomain("near(x[0], 0.1)")
left.mark(bm, 1)
right.mark(bm, 2)

mmHg2Pa = 132.32
freq = 1.0 
A = 2*mmHg2Pa
p_obs = Expression("A*sin(2*pi*f*t)", A=A,f=freq,t=0,degree=2)
n = FacetNormal(mesh)

g = Constant(0.0)
f = Constant([0.0])

In [ ]:
# total stress at both boundaries, no flux BC for pressure

boundary_conditions_u = {1:{"Neumann":n*p_obs},
                         2:{"Neumann":n*p_obs},
                        } 
                         
boundary_conditions_p = { 1:{"Neumann":Constant(0.0)},
                         2:{"Neumann":Constant(0.0)},
                         }
u_nullspace = True

solution = solve_biot(mesh, f, g, T, num_steps, material_parameter,
                      bm, boundary_conditions_p,
                      bm, boundary_conditions_u,
                      u_nullspace=u_nullspace, theta=1.0)
solution = [s.copy() for s in solution]

In [ ]:
V = FunctionSpace(mesh, "CG", 1)
displ = [s.split()[0] for s in solution]
total_pressure = [s.split()[1] for s in solution]
pressure = [s.split()[2] for s in solution]
lmbda_div_u = [project(material_parameter["lmbda"]*div(u), V) for u in displ]

x_coords = np.linspace(0.0, 0.1, 100)
slice_points = [Point(x, 0.0) for x in x_coords]

In [ ]:
pressures = {"negative_total_pressure" : extract_cross_section(total_pressure, slice_points)/mmHg2Pa,
             "fluid_pressure" : extract_cross_section(pressure, slice_points)/mmHg2Pa,
             "lambda_div_u"   : extract_cross_section(lmbda_div_u, slice_points)/mmHg2Pa,
             "prescribed outer pressure"   : extract_cross_section(p_obs, slice_points, times)/mmHg2Pa,}

displacement = {"displacement [m]": extract_cross_section(displ, slice_points),}
style_dict["negative_total_pressure"] = {"ls":":", "lw":3, "color":"firebrick"}
style_dict["fluid_pressure"] = {"ls":":", "lw":3, "color":"orange"}
style_dict["displacement [m]"] = {"ls":"-.", "lw":3, "color":"green"}
style_dict["p_opt"] = {"ls":"-.", "lw":3, "color":"blue"}
style_dict["prescribed outer pressure"] = {"ls":"-.", "lw":1, "color":"navy"}


In [ ]:
for i in [0, 4, 8, 12, 16]: #range(num_steps): 
    plot_pressures_and_forces_cross_section(pressures, displacement, i, x_coords)
    plt.suptitle(f"t = {times[i]:.3f} s")

In [ ]:
for i in [20, 40 ,60, 80]:
    plot_pressures_and_forces_timeslice(pressures, displacement, i, times)
    plt.suptitle(f"Point: ({slice_points[i].x():.3f}, {slice_points[i].y():.3f})")

In [ ]:
# total stress at both displacememt boundaries, no flux (right) and Dirichlet (left) for pressure

boundary_conditions_u = { 1:{"Neumann":n*p_obs},
                          2:{"Neumann":n*p_obs},
                        }

boundary_conditions_p = {1:{"Dirichlet":p_obs},
                         2:{"Neumann":Constant(0.0)},
                         }
u_nullspace = True
source_solution = solve_biot(mesh, f, g, T, num_steps, material_parameter,
                      bm, boundary_conditions_p,
                      bm, boundary_conditions_u,
                      u_nullspace=u_nullspace, theta=1.0)
source_solution = [s.copy() for s in source_solution]

In [ ]:
displ = [s.split()[0] for s in source_solution]
total_pressure = [s.split()[1] for s in source_solution]
pressure = [s.split()[2] for s in source_solution]
lmbda_div_u = [project(material_parameter["lmbda"]*div(u), V) for u in displ]

pressures = {"negative_total_pressure" : extract_cross_section(total_pressure, slice_points)/mmHg2Pa,
             "fluid_pressure" : extract_cross_section(pressure, slice_points)/mmHg2Pa,
             "lambda_div_u"   : extract_cross_section(lmbda_div_u, slice_points)/mmHg2Pa,
             "prescribed outer pressure"   : extract_cross_section(p_obs, slice_points, times)/mmHg2Pa,}

displacement = {"displacement [m]": extract_cross_section(displ, slice_points),}

for i in [0, 4, 8, 12, 16]: #range(num_steps): 
    plot_pressures_and_forces_cross_section(pressures, displacement, i, x_coords)
    plt.suptitle(f"t = {times[i]:.5f} s")

In [ ]:
for i in [20, 40 ,60, 80]:
    plot_pressures_and_forces_timeslice(pressures, displacement, i, times)
    plt.suptitle(f"Point: ({slice_points[i].x():.3f}, {slice_points[i].y():.3f})")

In [ ]:
# linear slope by different pressures in total stress left and right
p_obs_lin = Expression("A*sin(2*pi*f*t)*(0.8 + 2*x[0])", A=A,f=freq,t=0,degree=2)

boundary_conditions_u = {1:{"Neumann":n*p_obs_lin},
                         2:{"Neumann":n*p_obs_lin},
                        } 
                         
boundary_conditions_p = {1:{"Neumann":Constant(0.0)},
                         2:{"Neumann":Constant(0.0)},
                         }
u_nullspace = True
source_solution = solve_biot(mesh, f, g, T, num_steps, material_parameter,
                      bm, boundary_conditions_p,
                      bm, boundary_conditions_u,
                      u_nullspace=u_nullspace, theta=1.0)
source_solution = [s.copy() for s in source_solution]
displ = [s.split()[0] for s in source_solution]
total_pressure = [s.split()[1] for s in source_solution]
pressure = [s.split()[2] for s in source_solution]
lmbda_div_u = [project(material_parameter["lmbda"]*div(u), V) for u in displ]

pressures = {"negative_total_pressure" : extract_cross_section(total_pressure, slice_points)/mmHg2Pa,
             "fluid_pressure" : extract_cross_section(pressure, slice_points)/mmHg2Pa,
             "lambda_div_u"   : extract_cross_section(lmbda_div_u, slice_points)/mmHg2Pa,
             "prescribed outer pressure"   : extract_cross_section(p_obs_lin, slice_points, times)/mmHg2Pa,}

displacement = {"displacement [m]": extract_cross_section(displ, slice_points),}

for i in [0, 4, 8, 12, 16]: #range(num_steps): 
    plot_pressures_and_forces_cross_section(pressures, displacement, i, x_coords)
    plt.suptitle(f"t = {times[i]:.5f} s")

In [ ]:
# point source term and no displacement BCs (leading to strong uniform pressure increase)
g = Expression("A*exp(-(x[0] - 0.05)*(x[0] - 0.05)/(2*sig*sig))*sin(2*pi*t)", A=A, sig=0.01,t=0, degree=4)

boundary_conditions_u = { 1:{"Dirichlet":Constant([0.0])},
                          2:{"Dirichlet":Constant([0.0])},
                        }

boundary_conditions_p = {1:{"Neumann":Constant(0.0)},
                         2:{"Neumann":Constant(0.0)},
                         }
u_nullspace = False
source_solution = solve_biot(mesh, f, g, T, num_steps, material_parameter,
                      bm, boundary_conditions_p,
                      bm, boundary_conditions_u,
                      u_nullspace=u_nullspace, theta=1.0)
source_solution = [s.copy() for s in source_solution]
displ = [s.split()[0] for s in source_solution]
total_pressure = [s.split()[1] for s in source_solution]
pressure = [s.split()[2] for s in source_solution]
lmbda_div_u = [project(material_parameter["lmbda"]*div(u), V) for u in displ]

pressures = {"negative_total_pressure" : extract_cross_section(total_pressure, slice_points)/mmHg2Pa,
             "fluid_pressure" : extract_cross_section(pressure, slice_points)/mmHg2Pa,
             "lambda_div_u"   : extract_cross_section(lmbda_div_u, slice_points)/mmHg2Pa,
            }
displacement = {"displacement [m]": extract_cross_section(displ, slice_points),}

for i in [0, 4, 8, 12, 16]: #range(num_steps): 
    plot_pressures_and_forces_cross_section(pressures, displacement, i, x_coords)
    plt.suptitle(f"t = {times[i]:.5f} s")

In [ ]:
    
for i in [20, 40 ,60, 80]:
    plot_pressures_and_forces_timeslice(pressures, displacement, i, times)
    plt.suptitle(f"Point: ({slice_points[i].x():.3f}, {slice_points[i].y():.3f})")